In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pandas_datareader as data
import yfinance as yf

In [ ]:
start = '2010-01-01'
end = '2025-01-01'

# Download data using yfinance
df = yf.download('TSLA', start=start, end=end)

# Print the head of the DataFrame
print(df.head())

In [ ]:
df.tail()

In [ ]:
df  = df.reset_index()
df.head()

In [ ]:
df.info()

In [ ]:
plt.plot(df.Close)

In [ ]:
plt.plot(df.Open)

In [ ]:
plt.plot(df.Volume)

In [ ]:
mov_avg_100 = df.Close.rolling(100).mean()
mov_avg_100

In [ ]:
plt.figure(figsize=(12,6))
plt.plot(df.Close)
plt.plot(mov_avg_100, 'r')

In [ ]:
mov_avg_200 = df.Close.rolling(200).mean()
mov_avg_200

In [ ]:
plt.plot(df.Close)
plt.plot(mov_avg_100, 'r')
plt.plot(mov_avg_200, 'g')

In [ ]:
df.shape

In [ ]:
#splitting data
data_training = pd.DataFrame(df['Close'][0:int(len(df)*0.70)])
data_testing = pd.DataFrame(df['Close'][int(len(df)*0.70):int(len(df))])

print(data_training.shape)
print(data_testing.shape)

In [ ]:
data_testing.head()

In [ ]:
data_training.head()

In [ ]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler(feature_range=(0,1))

In [ ]:
data_training_array = scaler.fit_transform(data_training)
data_training_array

In [ ]:
#ML MODEL

In [ ]:
data_training_array.shape

In [ ]:
x_train =[]
y_train =[]

for i in range(100,data_training_array.shape[0]):
  x_train.append(data_training_array[i-100:i])
  y_train.append(data_training_array[i,0])

x_train,y_train =np.array(x_train),np.array(y_train)

In [ ]:
x_train.shape

In [ ]:
from keras.layers import Dense,Dropout,LSTM
from keras.models import Sequential

In [ ]:
model = Sequential()
model.add(LSTM(units=50,activation='relu',return_sequences=True,input_shape=(x_train.shape[1],1)))
model.add(Dropout(0.2))

model.add(LSTM(units=60,activation='relu',return_sequences=True))
model.add(Dropout(0.3))

model.add(LSTM(units=80,activation='relu',return_sequences=True))
model.add(Dropout(0.3))

model.add(LSTM(units=120,activation='relu'))
model.add(Dropout(0.5))

model.add(Dense(units=1))

In [ ]:
model.summary()

In [ ]:
model.compile(optimizer='adam',loss='mean_squared_error')
model.fit(x_train,y_train,epochs=50)

In [ ]:
model.save('keras_model.h5')

In [ ]:
data_testing.head()

In [ ]:
data_testing.tail()

In [ ]:
past_100_days = data_training.tail(100)

In [ ]:
# Instead of using append, use concat to combine the DataFrames:
final_df = pd.concat([past_100_days, data_testing], ignore_index=True)

In [ ]:
final_df.head()

In [ ]:
input_data = scaler.fit_transform(final_df)
input_data

In [ ]:
input_data.shape

In [ ]:
x_test = []
y_test = []

for i in range(100,input_data.shape[0]):
  x_test.append(input_data[i-100: i])
  y_test.append(input_data[i,0])

In [ ]:
x_test, y_test = np.array(x_test), np.array(y_test)
print(x_test.shape)
print(y_test.shape)

In [ ]:
y_predicted = model.predict(x_test)

In [ ]:
y_predicted.shape

In [ ]:
y_test

In [ ]:
y_predicted

In [ ]:
scaler.scale_

In [ ]:
scaler_factor = 1/0.02099517
y_predicted = y_predicted * scaler_factor
y_test = y_test * scaler_factor

In [ ]:
plt.figure(figsize=(12,6))
plt.plot(y_test, 'b', label = 'Original Price')
plt.plot(y_predicted, 'r', label = 'Predicted Price')
plt.xlabel('Time')
plt.ylabel('Price')
plt.legend()
plt.show()